In [2]:
!pip install transformers torch spacy sentencepiece
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.4 MB/s eta 0:00:00
2023-12-10 09:53:54.526711: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 09:53:54.526771: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 09:53:54.526810: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 09:53:54.535059: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the 

In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, MarianMTModel, MarianTokenizer
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from torch.nn import BCEWithLogitsLoss
import spacy
import random
import pandas as pd

In [4]:
class BertPreprocessor:
    def __init__(self, model_name='bert-base-uncased', max_length=256):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.max_length = max_length

    def preprocess(self, texts):
        encoding = self.tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            max_length=self.max_length,
            return_attention_mask=True,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return encoding['input_ids'], encoding['attention_mask']

In [5]:
class BackTranslationAugmentor:
    def __init__(self, src_lang="en", target_lang="fr"):
        self.src_lang = src_lang
        self.target_lang = target_lang
        self.src_model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{target_lang}'
        self.target_model_name = f'Helsinki-NLP/opus-mt-{target_lang}-{src_lang}'
        self.src_tokenizer = MarianTokenizer.from_pretrained(self.src_model_name)
        self.target_tokenizer = MarianTokenizer.from_pretrained(self.target_model_name)
        self.src_model = MarianMTModel.from_pretrained(self.src_model_name)
        self.target_model = MarianMTModel.from_pretrained(self.target_model_name)

    def translate(self, texts, model, tokenizer):
        # Ensure texts is a list
        if isinstance(texts, str):
            texts = [texts]
        # Tokenize the texts and handle potential long sequences
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        translated = model.generate(**inputs)
        return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    def back_translate(self, text):
        translated_text = self.translate(text, self.src_model, self.src_tokenizer)
        back_translated_text = self.translate(translated_text, self.target_model, self.target_tokenizer)
        return back_translated_text[0]

In [7]:
class BertModel:
    def __init__(self, model_name='bert-base-uncased'):
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def train(self, input_ids, attention_masks, labels, batch_size=16, epochs=4):
        dataset = TensorDataset(input_ids, attention_masks, labels)
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        optimizer = AdamW(self.model.parameters(), lr=2e-5)
        loss_fct = BCEWithLogitsLoss()

        for epoch in range(epochs):
            self.model.train()
            total_loss = 0
            for batch in train_dataloader:
                b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
                self.model.zero_grad()

                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                logits = outputs.logits

                b_labels = b_labels.unsqueeze(1)
                b_labels = torch.cat((1 - b_labels, b_labels), dim=1)

                loss = loss_fct(logits, b_labels.float())
                total_loss += loss.item()
                loss.backward()
                optimizer.step()

            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.2f}")

            self.model.eval()
            total_eval_accuracy = 0
            for batch in val_dataloader:
                b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
                with torch.no_grad():
                    outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                logits = outputs.logits
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                predictions = np.argmax(logits, axis=1)
                total_eval_accuracy += accuracy_score(label_ids, predictions)

            avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
            print(f"Validation Accuracy: {avg_val_accuracy:.2f}")

    def evaluate(self, input_ids, attention_masks, labels, batch_size=16):
        test_dataset = TensorDataset(input_ids, attention_masks, labels)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        self.model.eval()
        total_eval_accuracy = 0
        predictions, true_labels = [], []

        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
            with torch.no_grad():
                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            batch_predictions = np.argmax(logits, axis=1)
            predictions.extend(batch_predictions)
            true_labels.extend(label_ids)

        print("Accuracy:", accuracy_score(true_labels, predictions))
        print("Classification Report:")
        print(classification_report(true_labels, predictions))

In [8]:
from google.colab import files
uploaded = files.upload()

Saving test_data.tsv to test_data.tsv
Saving train_data.tsv to train_data.tsv
Saving Validation_data.tsv to Validation_data.tsv


In [9]:
def main():
    # Load the dataset
    df1 = pd.read_csv('train_data.tsv', sep='\t')
    df2 = pd.read_csv('test_data.tsv', sep='\t')
    df3 = pd.read_csv('Validation_data.tsv', sep='\t')

    # Define the column names
    columns = ['index', 'id', 'label', 'statement', 'subject', 'speaker', 'JobTitle', 'State', 'Party', 'BTC', 'FC', 'HT', 'MT', 'POF', 'context', 'justification']
    df1.columns = columns
    df2.columns = columns
    df3.columns = columns

    # Concatenate the dataframes
    df = pd.concat([df1, df2, df3]).reset_index(drop=True)

    # Map labels to binary values
    label_mapping = {
        'true': 1, 'half-true': 1, 'mostly-true': 1,
        'false': 0, 'pants-fire': 0, 'barely-true': 0
    }
    df['label'] = df['label'].map(label_mapping)

    # Initialize back translation augmentor
    augmentor = BackTranslationAugmentor()

    # Randomly select a subset of data for augmentation
    augmentation_indices = random.sample(range(len(df)), int(len(df) * 0.1))
    augmented_rows = []
    count = 0
    for idx in augmentation_indices:
        count = count + 1
        original_text = df.loc[idx, 'statement']
        augmented_text = augmentor.back_translate(original_text)
        augmented_rows.append({'statement': augmented_text, 'label': df.loc[idx, 'label']})

        # Print some examples of original and augmented texts
        if count < 5:
            print("Original:", original_text)
            print("Augmented:", augmented_text)
            print("----------")


    df_augmented = pd.DataFrame(augmented_rows)
    df = pd.concat([df, df_augmented], ignore_index=True)

    bert_preprocessor = BertPreprocessor()
    texts = df['statement'].tolist()
    input_ids, attention_masks = bert_preprocessor.preprocess(texts)

    y_bert = torch.tensor(df['label'].values)

    # Train and evaluate the BERT model
    bert_model = BertModel()
    bert_model.train(input_ids, attention_masks, y_bert)
    print("Finished Training, beginning Evaluation")
    bert_model.evaluate(input_ids, attention_masks, y_bert)

main()


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4 - Loss: 0.66
Validation Accuracy: 0.64
Epoch 2/4 - Loss: 0.58
Validation Accuracy: 0.67
Epoch 3/4 - Loss: 0.37
Validation Accuracy: 0.66
Epoch 4/4 - Loss: 0.15
Validation Accuracy: 0.66
Finished Training, beginning Evaluation
Accuracy: 0.9498791411915256
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.91      0.94      6211
           1       0.93      0.98      0.96      7855

    accuracy                           0.95     14066
   macro avg       0.95      0.95      0.95     14066
weighted avg       0.95      0.95      0.95     14066

